In [2]:
import numpy as np
import pickle, os

In [2]:
with open("data/processed/2025_1Q_LGES_Audit_Report_CONFS_en_fulltext.txt", 'r') as f:
    lges_2025_1q_fulltext = f.read()

In [1]:
with open("data/processed/2025_2Q_LGES_audit_report_summary.txt", 'r') as f:
    lges_2025_2q_summ = f.read()

with open("data/processed/2025_09_11_LGES_news_summary.txt", "r") as f:
    lges_2025_09_01_news_summary = f.read()

In [5]:
all_about_lges_text = lges_2025_2q_summ + "\n"+ lges_2025_09_01_news_summary

# Building synthetic dataset

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

chunks = text_splitter.create_documents([all_about_lges_text])

# docs_processed = []
# for doc in langchain_docs:
#     docs_processed += text_splitter.split_documents([doc])

In [13]:
len(chunks)

57